In [1]:
# This notebook is for pretraining and finetuning models... (RESNET)

In [2]:
# Importing the necessary libraries #
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms.functional as F
import lightning as L
from typing import List
from lightning.pytorch import Trainer
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import precision_score
from torchmetrics import Accuracy
import utils
import os

In [3]:
torch.cuda.empty_cache()
torch.set_float32_matmul_precision("medium")

In [4]:
import wandb
from lightning.pytorch.loggers import WandbLogger
wandb.login()
# Initializing wandb logger #
wandb_logger = WandbLogger(
    entity="A2_DA6401_DL",
    project="ResNet",       
)
wandb.login(key="5ef7c4bbfa350a2ffd3c198cb9289f544e3a0910")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ae21b105 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/joel/.netrc


True

In [ ]:
import torch.nn as nn
from torchvision import models
from torchvision.models import ResNet50_Weights

# Load pretrained ResNet-50
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.BatchNorm1d(1024),
    nn.Mish(),
    nn.Dropout(0.4),
    nn.Linear(1024, 10)
)

# Unfreeze 
for param in model.fc.parameters():
    param.requires_grad = True

"""
for param in model.layer4.parameters():
    param.requires_grad = True
"""

print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# Defining the dataloaders to be built
train_dataset, val_dataset, data_transforms = utils.create_dataset_image_folder(path_=os.path.join(os.path.abspath(""), "nature_12K/inaturalist_12K/train/"), input_size=(224,224))
train_loader, val_loader = utils.create_dataloaders(batch_size=32, num_workers=2, train_dataset=train_dataset, val_dataset=val_dataset, is_data_aug=True, data_transforms = data_transforms)

# Setting up the callbacks to be used
early_stopping = EarlyStopping('val_acc', patience=5, mode="max")
checkpoint_callback = ModelCheckpoint(monitor="val_acc", dirpath="checkpoints/", filename="best-checkpoint_resnet_head", save_top_k=1, mode="max")
# Defining the model
lit_model = utils.Lightning_CNN(model = model)
# Training the Model
trainer = Trainer(max_epochs=50, precision="16-mixed", accelerator="auto", logger=wandb_logger, callbacks=[early_stopping, checkpoint_callback])
trainer.fit(lit_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


/home/joel/Pytorch_CUDA/virt_env/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/joel/Pytorch_CUDA/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | model      | ResNet             | 25.6 M | train
1 | loss_fn    | CrossEntropyLoss   | 0      | train
2 | acc_metric | MulticlassAccuracy | 0      | train
----------------------------------------------------------
2.1 M     Trainable params
23.5 M    Non-trainable params
25.6 M    Total params
102.474   Total estimated model params size (MB)
158       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [7]:
# Recalling the model weights from the callbacks
best_model_path = checkpoint_callback.best_model_path
lit_model = utils.Lightning_CNN.load_from_checkpoint(best_model_path, model=model)

# Getting the test dataloader
test_loader = utils.get_test_dataloader(os.path.join(os.path.abspath(""), "nature_12K/inaturalist_12K/val/"), data_transforms)
# Prediction of the test data
trainer = Trainer(logger=wandb_logger)
# Running prediction
predictions = trainer.test(model=lit_model, dataloaders=test_loader)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7429999709129333     │
│         test_loss         │    0.7525216937065125     │
└───────────────────────────┴───────────────────────────┘